In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
#importing Audio Visualisation Libraries
from librosa import display
import librosa
#Getting the csv as dataframe using pandas
data=pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")

In [2]:
#displaying the structure of the CSV
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [3]:
#preprocessing using all features set
x_train=[]
x_test=[]
y_train=[]
y_test=[]
path="UrbanSound8K/audio/fold"
for i in tqdm(range(len(data))):
    fold_no=str(data.iloc[i]["fold"])
    file=data.iloc[i]["slice_file_name"]
    label=data.iloc[i]["classID"]
    filename=path+fold_no+"/"+file
    y,sr=librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cq = np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=40).T,axis=0)
    chroma_cens = np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=40).T,axis=0)
    features=np.reshape(np.vstack((mfccs,melspectrogram,chroma_stft,chroma_cq,chroma_cens)),(40,5))
    if(fold_no!='10'):
      x_train.append(features)
      y_train.append(label)
    else:
      x_test.append(features)
      y_test.append(label)

  1%|▌                                                                             | 67/8732 [00:30<1:06:07,  2.18it/s]C:\Anaconda3\lib\site-packages\librosa\core\pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
100%|████████████████████████████████████████████████████████████████████████████| 8732/8732 [1:04:30<00:00,  2.83it/s]


In [4]:
#converting the lists into numpy arrays
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((7895, 40, 5), (837, 40, 5), (7895,), (837,))

In [5]:
#reshaping into 2d to save in csv format
x_train_2d=np.reshape(x_train,(x_train.shape[0],x_train.shape[1]*x_train.shape[2]))
x_test_2d=np.reshape(x_test,(x_test.shape[0],x_test.shape[1]*x_test.shape[2]))
x_train_2d.shape,x_test_2d.shape

((7895, 200), (837, 200))

In [6]:
#saving the data numpy arrays
np.savetxt("train_data_multi.csv", x_train_2d, delimiter=",")
np.savetxt("test_data_multi.csv",x_test_2d,delimiter=",")
np.savetxt("train_labels_multi.csv",y_train,delimiter=",")
np.savetxt("test_labels_multi.csv",y_test,delimiter=",")

In [16]:
#extracting data from csv files into numpy arrays
from numpy import genfromtxt
x_train = genfromtxt('train_data_multi.csv', delimiter=',')
y_train = genfromtxt('train_labels_multi.csv', delimiter=',')
x_test = genfromtxt('test_data_multi.csv', delimiter=',')
y_test = genfromtxt('test_labels_multi.csv', delimiter=',')

In [17]:
#shape
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((7895, 200), (837, 200), (7895,), (837,))

In [18]:
#converting to one hot
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
y_train.shape,y_test.shape

((7895, 10), (837, 10))

In [19]:
#shapes
x_train.shape,x_test.shape

((7895, 200), (837, 200))

In [20]:
from keras import Sequential
from keras.layers import Dense,Dropout,Activation

In [21]:
#forming model
model=Sequential()
#building the model
model.add(Dense(units=256,activation='relu',input_dim=200))
model.add(Dropout(0.4))
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=10,activation='softmax'))

In [22]:
#compiling
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
#fitting
model.fit(x_train,y_train,epochs=30,validation_data=(x_test,y_test),batch_size=50)

Train on 7895 samples, validate on 837 samples
Epoch 1/30
7895/7895 [==============================] - ETA: 1s - loss: 0.4582 - acc: 0.820 - ETA: 0s - loss: 0.3905 - acc: 0.870 - ETA: 0s - loss: 0.4095 - acc: 0.854 - ETA: 0s - loss: 0.4175 - acc: 0.854 - ETA: 0s - loss: 0.4064 - acc: 0.861 - ETA: 0s - loss: 0.4264 - acc: 0.855 - ETA: 0s - loss: 0.4342 - acc: 0.852 - ETA: 0s - loss: 0.4427 - acc: 0.851 - ETA: 0s - loss: 0.4325 - acc: 0.856 - ETA: 0s - loss: 0.4299 - acc: 0.856 - ETA: 0s - loss: 0.4279 - acc: 0.856 - ETA: 0s - loss: 0.4276 - acc: 0.855 - ETA: 0s - loss: 0.4349 - acc: 0.854 - ETA: 0s - loss: 0.4402 - acc: 0.853 - ETA: 0s - loss: 0.4426 - acc: 0.853 - ETA: 0s - loss: 0.4410 - acc: 0.853 - ETA: 0s - loss: 0.4398 - acc: 0.853 - ETA: 0s - loss: 0.4427 - acc: 0.852 - 1s 122us/step - loss: 0.4408 - acc: 0.8529 - val_loss: 1.2953 - val_acc: 0.6607
Epoch 2/30
7895/7895 [==============================] - ETA: 1s - loss: 0.5459 - acc: 0.840 - ETA: 0s - loss: 0.5000 - acc: 0.835 - E

7895/7895 [==============================] - ETA: 1s - loss: 0.5554 - acc: 0.880 - ETA: 1s - loss: 0.3854 - acc: 0.880 - ETA: 0s - loss: 0.3475 - acc: 0.878 - ETA: 0s - loss: 0.3613 - acc: 0.880 - ETA: 0s - loss: 0.3558 - acc: 0.879 - ETA: 0s - loss: 0.3519 - acc: 0.876 - ETA: 0s - loss: 0.3557 - acc: 0.873 - ETA: 0s - loss: 0.3591 - acc: 0.873 - ETA: 0s - loss: 0.3613 - acc: 0.876 - ETA: 0s - loss: 0.3543 - acc: 0.877 - ETA: 0s - loss: 0.3562 - acc: 0.877 - ETA: 0s - loss: 0.3544 - acc: 0.878 - ETA: 0s - loss: 0.3609 - acc: 0.876 - ETA: 0s - loss: 0.3608 - acc: 0.877 - ETA: 0s - loss: 0.3690 - acc: 0.876 - ETA: 0s - loss: 0.3690 - acc: 0.876 - ETA: 0s - loss: 0.3704 - acc: 0.877 - ETA: 0s - loss: 0.3730 - acc: 0.875 - 1s 123us/step - loss: 0.3725 - acc: 0.8766 - val_loss: 1.3952 - val_acc: 0.6476
Epoch 11/30
7895/7895 [==============================] - ETA: 1s - loss: 0.4160 - acc: 0.880 - ETA: 0s - loss: 0.3758 - acc: 0.880 - ETA: 0s - loss: 0.3795 - acc: 0.868 - ETA: 0s - loss: 0.38

7895/7895 [==============================] - ETA: 1s - loss: 0.5446 - acc: 0.840 - ETA: 0s - loss: 0.3305 - acc: 0.895 - ETA: 0s - loss: 0.3627 - acc: 0.891 - ETA: 0s - loss: 0.3802 - acc: 0.884 - ETA: 0s - loss: 0.3716 - acc: 0.883 - ETA: 0s - loss: 0.3733 - acc: 0.881 - ETA: 0s - loss: 0.3560 - acc: 0.885 - ETA: 0s - loss: 0.3559 - acc: 0.884 - ETA: 0s - loss: 0.3497 - acc: 0.883 - ETA: 0s - loss: 0.3429 - acc: 0.885 - ETA: 0s - loss: 0.3401 - acc: 0.886 - ETA: 0s - loss: 0.3354 - acc: 0.888 - ETA: 0s - loss: 0.3326 - acc: 0.890 - ETA: 0s - loss: 0.3362 - acc: 0.889 - ETA: 0s - loss: 0.3370 - acc: 0.891 - ETA: 0s - loss: 0.3331 - acc: 0.892 - ETA: 0s - loss: 0.3317 - acc: 0.892 - ETA: 0s - loss: 0.3278 - acc: 0.894 - ETA: 0s - loss: 0.3316 - acc: 0.893 - 1s 126us/step - loss: 0.3293 - acc: 0.8942 - val_loss: 1.3314 - val_acc: 0.6487
Epoch 20/30
7895/7895 [==============================] - ETA: 1s - loss: 0.3116 - acc: 0.880 - ETA: 1s - loss: 0.2812 - acc: 0.886 - ETA: 0s - loss: 0.30

7895/7895 [==============================] - ETA: 1s - loss: 0.4663 - acc: 0.860 - ETA: 1s - loss: 0.3202 - acc: 0.888 - ETA: 1s - loss: 0.2972 - acc: 0.896 - ETA: 0s - loss: 0.3118 - acc: 0.891 - ETA: 0s - loss: 0.2973 - acc: 0.898 - ETA: 0s - loss: 0.3074 - acc: 0.898 - ETA: 0s - loss: 0.3175 - acc: 0.895 - ETA: 0s - loss: 0.3164 - acc: 0.896 - ETA: 0s - loss: 0.3149 - acc: 0.897 - ETA: 0s - loss: 0.3163 - acc: 0.897 - ETA: 0s - loss: 0.3137 - acc: 0.898 - ETA: 0s - loss: 0.3154 - acc: 0.897 - ETA: 0s - loss: 0.3106 - acc: 0.899 - ETA: 0s - loss: 0.3127 - acc: 0.899 - ETA: 0s - loss: 0.3100 - acc: 0.900 - ETA: 0s - loss: 0.3073 - acc: 0.902 - ETA: 0s - loss: 0.3031 - acc: 0.902 - ETA: 0s - loss: 0.3057 - acc: 0.902 - ETA: 0s - loss: 0.3050 - acc: 0.901 - ETA: 0s - loss: 0.3086 - acc: 0.901 - 1s 138us/step - loss: 0.3067 - acc: 0.9025 - val_loss: 1.5335 - val_acc: 0.6404
Epoch 29/30
7895/7895 [==============================] - ETA: 1s - loss: 0.2523 - acc: 0.880 - ETA: 1s - loss: 0.24

In [24]:
model.summary()

# Calculate pre-training accuracy 
score_train = model.evaluate(x_train, y_train, verbose=1)
score_test = model.evaluate(x_test, y_test, verbose=1)
accuracy_train = 100*score_train[1]
accuracy_test = 100*score_test[1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               51456     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2570      
Total para

In [25]:
print("Training accuracy: %.4f%%" % accuracy_train)
print("Test accuracy: %.4f%%" % accuracy_test)

Training accuracy: 93.2489%
Test accuracy: 62.8435%
